<i>Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.</i>
<br><br>
# Recommender Hyperparameter Tuning w/ AzureML

This notebook shows how to auto-tune hyperparameters of a recommender model by utilizing **Azure Machine Learning service**<sup>[a](#azureml-search), [b](#azure-subscription)</sup> ([AzureML](https://azure.microsoft.com/en-us/services/machine-learning-service/)).

We present an overall process of utilizing AzureML, specifically [**Hyperdrive**](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py) component, for the hyperparameter tuning by demonstrating key steps:
1. Configure AzureML Workspace
2. Create Remote Compute Target (GPU cluster)
3. Prepare Data
4. Prepare Training Scripts
5. Setup and Run Hyperdrive Experiment
6. Model Import, Re-train and Test

In this notebook, we use [**wide-and-deep model**](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html) from **TensorFlow high-level Estimator API (v1.12)** on the movie recommendation scenario. Wide-and-deep learning jointly trains wide linear model and deep neural networks (DNN) to combine the benefits of memorization and generalization for recommender systems.

For more details about the **wide-and-deep** model:
* [Wide-Deep Quickstart notebook](../00_quick_start/wide_deep_model_movielens.ipynb)
* [Original paper](https://arxiv.org/abs/1606.07792)
* [TensorFlow API doc](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNLinearCombinedRegressor)
  
Regarding **AuzreML**, please refer:
* [Quickstart notebook](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)
* [Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)
* [Tensorflow model tuning with Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-tensorflow)

> <span id="azureml-search">a. </span>To use AzureML, you will need an Azure subscription.  
<span id="azure-subscription">b. </span>When you web-search "Azure Machine Learning", you will most likely to see mixed results of Azure Machine Learning (AzureML) and Azure Machine Learning **Studio**. Please note they are different services where AzureML's focuses are on ML model management, tracking and hyperparameter tuning, while the [ML Studio](https://studio.azureml.net/)'s is to provide a high-level tool for 'easy-to-use' experience of ML designing and experimentation based on GUI.   

In [ ]:
# AzureML workspace info. Note, will look up "aml_config\config.json" first, then fall back to use this
SUBSCRIPTION_ID = '<subscription-id>'
RESOURCE_GROUP  = '<resource-group>'
WORKSPACE_NAME  = '<workspace-name>'

# Remote compute (cluster) configuration. If you want to save the cost more, set these to small.
VM_SIZE = 'STANDARD_NC6'
VM_PRIORITY = 'lowpriority'
# Cluster nodes
MIN_NODES = 4
MAX_NODES = 8
# Hyperdrive experimentation configuration
MAX_TOTAL_RUNS = 200  # Number of runs (training-and-evaluation) to search the best hyperparameters. 
MAX_CONCURRENT_RUNS = 8

# Recommend top k items
TOP_K = 10
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'
# Number of samples to train. Epochs = NUM_SAMPLES_TO_TRAIN / len(train)
NUM_SAMPLES_TO_TRAIN = 20000000
# Metrics to track. Put your primary metric (to search) at the first place
METRICS = ['rmse', 'ndcg']  
# Data column names
USER_COL = 'UserId'
ITEM_COL = 'MovieId'
RATING_COL = 'Rating'
ITEM_FEAT_COL = 'Genres'

### 1. Configure AzureML Workspace
**AzureML workspace** is a foundational block in the cloud that you use to experiment, train, and deploy machine learning models via AzureML service. In this notebook, we 1) create a workspace from [**Azure portal**](https://portal.azure.com) and 2) configure from this notebook.

You can find more details about the setup and configure processes from the following links:
* [Quickstart with Azure portal](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started)
* [Quickstart with Python SDK](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)
  
<br>
  
#### 1.1 Create a workspace
1. Sign in to the [Azure portal](https://portal.azure.com) by using the credentials for the Azure subscription you use.
2. Select **Create a resource** menu, search for **Machine Learning service workspace** select **Create** button.
3. In the **ML service workspace** pane, configure your workspace with entering the *workspace name* and *resource group* (or **create new** resource group if you don't have one already), and select **Create**. It can take a few moments to create the workspace.
  
<br>
  
#### 1.2 Configure
To configure this notebook to communicate with the workspace, type in your Azure subscription id, the resource group name and workspace name to `<subscription-id>`, `<resource-group>`, `<workspace-name>` in the above notebook cell. Alternatively, you can create a *.\aml_config\config.json* file with the following contents:
```
{
    "subscription_id": "<subscription-id>",
    "resource_group": "<resource-group>",
    "workspace_name": "<workspace-name>"
}
```


Now let's see if everything is ready!

In [ ]:
import sys
sys.path.append("../../")

import os
import shutil
import itertools

import papermill as pm
import tensorflow as tf
import pandas as pd
import sklearn.preprocessing

import azureml as aml
import azureml.widgets as widgets
import azureml.train.hyperdrive as hd

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.common import tf_utils
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

from IPython.display import clear_output

print("Azure ML SDK Version:", aml.core.VERSION)
print("Tensorflow Version:", tf.__version__)

In [ ]:
# Connect to a workspace
try:
    ws = aml.core.Workspace.from_config()
except aml.exceptions.UserErrorException:
    try:
        ws = aml.core.Workspace(
            subscription_id=SUBSCRIPTION_ID,
            resource_group=RESOURCE_GROUP,
            workspace_name=WORKSPACE_NAME
        )
    except aml.exceptions.AuthenticationException:
        ws = None

if ws is None:
    raise ValueError(
        """Cannot access the AzureML workspace w/ the config info provided.
        Please check if you entered the correct id, group name and workspace name"""
    )
else:
    print("AzureML workspace name: ", ws.name)
    clear_output()  # Comment out this if you want to see your workspace info.

### 2. Create Remote Compute Target

We create a gpu cluster as our **remote compute target**. If a cluster with the same name is already exist in your workspace, the script will load it instead. You can see [this document](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up a compute target on different locations.

This notebook selects **STANDARD_NC6** virtual machine and sets it's priority as *lowpriority* to save the cost.

Size | vCPU | Memory (GiB) | Temp storage (SSD, GiB) | GPU | GPU memory (GiB) | Max data disks | Max NICs
---|---|---|---|---|---|---|---
Standard_NC6 | <center>6</center> | <center>56</center> | <center>340</center> | <center>1</center> | <center>8</center> | <center>24</center> | <center>1</center>

For more information about Azure virtual machine sizes, see [here](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-gpu).

In [ ]:
CLUSTER_NAME = 'gpu-cluster-nc6'

try:
    compute_target = aml.core.compute.ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing compute target")
except aml.core.compute_target.ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = aml.core.compute.AmlCompute.provisioning_configuration(
        vm_size=VM_SIZE,
        vm_priority=VM_PRIORITY,
        min_nodes=MIN_NODES,
        max_nodes=MAX_NODES
    )
    # create the cluster
    compute_target = aml.core.compute.ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

### 3. Prepare Data
For demonstration purpose, we use 1M MovieLens dataset. First, download the data and convert the format (multi-hot encode *genres*) to make it work for our model. More details about this step is described in our [Wide-Deep Quickstart notebook](../00_quick_start/wide_deep_model_movielens.ipynb).

In [ ]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=[USER_COL, ITEM_COL, RATING_COL],
    genres_col='Genres_string'
)

# Encode 'genres' into int array (multi-hot representation) to use as item features
genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
data[ITEM_FEAT_COL] = genres_encoder.fit_transform(
    data['Genres_string'].apply(lambda s: s.split("|"))
).tolist()
data.drop('Genres_string', axis=1, inplace=True)

data.head()

The dataset is split into train, validation, and test sets. The train and validation sets will be used for hyperparameter tuning, and the test set will be used for the final evaluation of the model after we import the best model from AzureML workspace.

Here, we don't use multiple-split directly by passing `ratio=[0.56, 0.19, 0.25]`. Instead, we first split the data into train and test sets with the same `seed` we've been using in other notebooks to make the train set identical across them. Then, we further split the train set into train and validation sets.

In [ ]:
# Use the same seed to make the train and test sets identical across other notebooks in the repo.
train, test = python_random_split(data, ratio=0.75, seed=123)
# Further split the train set into train and validation set.
train, valid = python_random_split(train)

print(len(train), len(valid), len(test))

Now, upload the train and validation sets to the AzureML workspace. Our Hyperdrivce experiment will use them.

In [ ]:
DATA_DIR = 'aml_data'

os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))
VALID_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_valid.pkl"
valid.to_pickle(os.path.join(DATA_DIR, VALID_FILE_NAME))

# Note, all the files under DATA_DIR will be uploaded to the data store
ds = ws.get_default_datastore()
ds.upload(
    src_dir=DATA_DIR,
    target_path='data',
    overwrite=True,
    show_progress=True
)

### 4. Prepare Training Scripts
Next step is to prepare scripts that AzureML Hyperdrive will use to train and evaluate models with selected hyperparameters. We re-use our [Wide-Deep Quickstart notebook](../00_quick_start/wide_deep_model_movielens.ipynb) for that. To run the model notebook from the Hyperdrive Run, all we need is to prepare an [entry script](../../reco_utils/aml/wide_deep.py) which parses the hyperparameter arguments, passes them to the notebook, and records the results of the notebook to AzureML Run logs by using `papermill`. Hyperdrive uses the logs to track the performance of each hyperparameter-set and finds the best performed one.  

Here is a code snippet from the [entry script](../../reco_utils/aml/wide_deep.py):
```
import argparse
import papermill as pm
from azureml.core import Run
run = Run.get_context()
...
parser = argparse.ArgumentParser()
...
parser.add_argument('--dnn-optimizer', type=str, dest='dnn_optimizer', ...
parser.add_argument('--dnn-optimizer-lr', type=float, dest='dnn_optimizer_lr', ...
...
pm.execute_notebook(
    "../../notebooks/00_quick_start/wide_deep_model_movielens.ipynb",
    OUTPUT_NOTEBOOK,
    parameters=params,
    kernel_name='python3',
)
...
```

In [ ]:
# Prepare all the necessary scripts which will be loaded to our Hyperdrive Experiment Run
SCRIPT_DIR = 'aml_script'

# Clean-up scripts if already exists
shutil.rmtree(SCRIPT_DIR, ignore_errors=True)

# Copy scripts to SCRIPT_DIR temporarly
shutil.copytree(os.path.join('..', '..', 'reco_utils'), os.path.join(SCRIPT_DIR, 'reco_utils'))

# We re-use our model notebook for training and testing models.
model_notebook_dir = os.path.join('notebooks', '00_quick_start')
dest_model_notebook_dir = os.path.join(SCRIPT_DIR, model_notebook_dir)
os.makedirs(dest_model_notebook_dir , exist_ok=True)
shutil.copy(
    os.path.join('..', '..', model_notebook_dir, 'wide_deep_model_movielens.ipynb'),
    dest_model_notebook_dir
)

# This is our entry script for Hyperdrive Run
ENTRY_SCRIPT_NAME = 'reco_utils/aml/wide_deep.py'

### 5. Setup and Run Hyperdrive Experiment
[Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) create a machine learning Experiment [Run](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run?view=azure-ml-py) on the workspace and utilizes child-runs to search the best set of hyperparameters.

<br>

#### 5.1 Create Experiment 
[Experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py) is the main entry point into experimenting with AzureML. To create new Experiment or get the existing one, we pass our experimentation name.

In [ ]:
# Create an experiment to track the runs in the workspace
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_wide_deep_model"
exp = aml.core.Experiment(workspace=ws, name=EXP_NAME)

#### 5.2 Define Search Space 
Now we define the search space of hyperparameters. For example, if you want to test different batch sizes of {64, 128, 256}, you can use `azureml.train.hyperdrive.choice(64, 128, 256)`. To search from a continuous space, use `uniform(start, end)`. For more options, see [Hyperdrive parameter expressions](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.parameter_expressions?view=azure-ml-py).
In this notebook, we fix model type as `wide_deep` and the number of samples to train as 20M. Note, 'the number of samples to train' is not the data size, but just a count of sample feeding.

FYI, epochs will be:
```
epochs = NUM_SAMPLES_TO_TRAIN / len(train)
```
and train steps will be:
```
train_steps = NUM_SAMPLES_TO_TRAIN / BATCH_SIZE
```

In the search space, we set different linear and DNN optimizers, structures, learning rates and regularization rates. Details about the hyperparameters can be found from our [Wide-Deep Quickstart notebook](../00_quick_start/wide_deep_model_movielens.ipynb).

> Hyperdrive provides three different parameter sampling methods: `RandomParameterSampling`, `GridParameterSampling`, and `BayesianParameterSampling`. Details about each method can be found from [Azure doc](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters). Here, we use the Bayesian sampling.

In [ ]:
# Fixed parameters
script_params = {
    '--datastore': ds.as_mount(),
    '--train-datapath': "data/" + TRAIN_FILE_NAME,
    '--test-datapath': "data/" + VALID_FILE_NAME,
    '--top-k': TOP_K,
    '--user-col': USER_COL,
    '--item-col': ITEM_COL,
    '--item-feat-col': ITEM_FEAT_COL,
    '--rating-col': RATING_COL,
    '--metrics': METRICS,
    '--num-samples-to-train': NUM_SAMPLES_TO_TRAIN,  # epochs = num-samples-to-train / num-train
    '--model-type': 'wide_deep'
}

# Hyperparameter search space
params = {
    '--batch-size': hd.choice(64, 128, 256),
    # Linear model hyperparameters
    '--linear-optimizer': hd.choice('Ftrl'),  # 'SGD' easily got exploded loss in regression problems.
    '--linear-optimizer-lr': hd.uniform(0.0001, 0.1),
    '--linear-l1-reg': hd.uniform(0.0, 0.1),
    # Deep model hyperparameters
    '--dnn-optimizer': hd.choice('Adagrad', 'Adam'),
    '--dnn-optimizer-lr': hd.uniform(0.0001, 0.1),
    '--dnn-user-embedding-dim': hd.choice(4, 8, 16, 32, 64, 128),
    '--dnn-item-embedding-dim': hd.choice(4, 8, 16, 32, 64, 128),
    '--dnn-hidden-layer-1': hd.choice(0, 32, 64, 128, 256, 512, 1024),  # 0: not using this layer
    '--dnn-hidden-layer-2': hd.choice(0, 32, 64, 128, 256, 512, 1024),
    '--dnn-hidden-layer-3': hd.choice(0, 32, 64, 128, 256, 512, 1024),
    '--dnn-hidden-layer-4': hd.choice(32, 64, 128, 256, 512, 1024),
    '--dnn-batch-norm': hd.choice(0, 1),
    '--dnn-dropout': hd.uniform(0.0, 0.5)
}

**AzureML Estimator** is the building block for training. An Estimator encapsulates the training code and parameters, the compute resources and runtime environment for a particular training scenario (Note, this is not TensorFlow's Estimator)

We create one for our experimentation with the dependencies our model requires as follows:
```
conda_packages=['pandas', 'scikit-learn', 'tensorflow-gpu=1.12'],
pip_packages=['ipykernel', 'papermill']
```

To the Hyperdrive Run Config, we set our primary metric name and the goal (our hyperparameter search criteria), hyperparameter sampling method, and number of total child-runs. The bigger the search space, the more number of runs we will need for better results.

In [ ]:
est = aml.train.estimator.Estimator(
    source_directory=SCRIPT_DIR,
    entry_script=ENTRY_SCRIPT_NAME,
    script_params=script_params,
    compute_target=compute_target,
    use_gpu=True,
    conda_packages=['pandas', 'scikit-learn', 'tensorflow-gpu=1.12'],
    pip_packages=['ipykernel', 'papermill']
)

hd_run_config = hd.HyperDriveRunConfig(
    estimator=est, 
    hyperparameter_sampling=hd.BayesianParameterSampling(params),
    primary_metric_name=METRICS[0],
    primary_metric_goal=hd.PrimaryMetricGoal.MINIMIZE, 
    max_total_runs=MAX_TOTAL_RUNS,
    max_concurrent_runs=MAX_CONCURRENT_RUNS
)

#### 5.3 Run Experiment

Now we submit the Run to our experiment. You can see the experiment progress from this notebook by using `azureml.widgets.RunDetails(hd_run).show()` or check from the Azure portal with the url link you can get by running `hd_run.get_portal_url()`.

![hyperdrive](hyperdrive.jpg)

<br>

To load an existing Hyperdrive Run, use `hd_run = hd.HyperDriveRun(exp, <user-run-id>, hyperdrive_run_config=hd_run_config)`. You also can cancel the Run with `hd_run.cancel()`.

In [ ]:
hd_run = exp.submit(config=hd_run_config)

In [ ]:
widgets.RunDetails(hd_run).show()

Once all the child-runs are finished, we can get the best run and the metrics.

In [ ]:
# Get best run and printout metrics
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

In [ ]:
display(best_run_metrics)

In [ ]:
print("* Best Run Id:", best_run.id)
print("\n* Best hyperparameters:")
print("Model type =", best_run_metrics['model_type'])
print("Batch size =", best_run_metrics['batch_size'])
print("Linear optimizer =", best_run_metrics['linear_optimizer'])
print("\tLearning rate = {0:.4f}".format(best_run_metrics['linear_optimizer_lr']))
print("\tL1 regularization = {0:.4f}".format(best_run_metrics['linear_l1_reg']))
print("DNN optimizer =", best_run_metrics['dnn_optimizer'])
print("\tUser embedding dimension =", best_run_metrics['dnn_user_dim'])
print("\tItem embedding dimension =", best_run_metrics['dnn_item_dim'])
print("\tHidden units =", best_run_metrics['dnn_hidden_units'])
print("\tLearning rate = {0:.4f}".format(best_run_metrics['dnn_optimizer_lr']))
print("\tDropout rate = {0:.4f}".format(best_run_metrics['dnn_dropout']))
print("\tBatch normalization =", best_run_metrics['dnn_batch_norm'])
print("\n* Performance metrics:")
print("Mean squared error = {0:.4f}".format(best_run_metrics['average_loss']))
for m in METRICS:
    print("\t{0} = {1:.4f}".format(m, best_run_metrics[m]))


### 6. Model Import, Re-train, and Test

[Wide-Deep Quickstart notebook](../00_quick_start/wide_deep_model_movielens.ipynb), which we've used in our Hyperdrive Experiment, exports the trained model to the output folder (the output path is recorded at `best_run_metrics['saved_model_dir']`). We can download a model from the best run and test it. 

In [ ]:
MODEL_DIR = 'aml_model'

os.makedirs(MODEL_DIR, exist_ok=True)
model_file_dir = os.path.normpath(best_run_metrics['saved_model_dir']) + '/'
for f in best_run.get_file_names():
    if f.startswith(model_file_dir):
        output_file_path = os.path.join(MODEL_DIR, f[len(model_file_dir):])
        print("Downloading {}..".format(f))
        best_run.download_file(name=f, output_file_path=output_file_path)

saved_model = tf.contrib.estimator.SavedModelEstimator(MODEL_DIR)

The validation dataset we split in the beginning has been used to evaluate the model during the hyperparameter tuning. Since we finished the tuning, now we use them to final train the model on top of what it already learned.

In [ ]:
# TODO test with and without this step.
train_fn = tf_utils.pandas_input_fn(
    df=valid,  # The set we have been used for hyperparameter tuning.
    y_col=RATING_COL,
    batch_size=BATCH_SIZE,
    num_epochs=None,  # None == run forever. We use steps=TRAIN_STEPS instead.
    shuffle=True
)

model.train(
    input_fn=train_fn,
    steps=1000  # We re-train a little bit more
)

Finally, let's test the model with the test set.

In [ ]:
cols = {
    'col_user': USER_COL,
    'col_item': ITEM_COL,
    'col_rating': RATING_COL,
    'col_prediction': 'prediction'
}

RANKING_METRICS = {
    'map': map_at_k,
    'ndcg': ndcg_at_k,
    'precision': precision_at_k,
    'recall': recall_at_k
}
RATING_METRICS = {
    'rmse': rmse,
    'mae': mae,
    'rsquared': rsquared,
    'exp_var': exp_var
}

# Prediction input function for TensorFlow SavedModel
def predict_input_fn(df):
    def input_fn():
        examples = [None] * len(df)
        for index, test_sample in df.iterrows():
            example = tf.train.Example()

            example.features.feature[USER_COL].int64_list.value.extend([test_sample[USER_COL]])
            example.features.feature[ITEM_COL].int64_list.value.extend([test_sample[ITEM_COL]])
            example.features.feature[ITEM_FEAT_COL].float_list.value.extend(test_sample[ITEM_FEAT_COL])

            examples[index] = example.SerializeToString()
        return {'inputs': tf.constant(examples)}
    return input_fn

In [ ]:
# Rating prediction set
X_test = test.drop(RATING_COL, axis=1).reset_index(drop=True)

# Rating prediction
predictions = list(itertools.islice(
    saved_model.predict(predict_input_fn(X_test)),
    len(X_test)
))
prediction_df = X_test.copy()
prediction_df['prediction'] = [p['predictions'][0] for p in predictions]
print(prediction_df['prediction'].describe(), "\n")
for m, fn in RATING_METRICS.items():
    result = fn(test, prediction_df, **cols)
    print(m, "=", result)

In [ ]:
# Unique items
if ITEM_FEAT_COL is None:
    items = data.drop_duplicates(ITEM_COL)[[ITEM_COL]].reset_index(drop=True)
else:
    items = data.drop_duplicates(ITEM_COL)[[ITEM_COL, ITEM_FEAT_COL]].reset_index(drop=True)
# Unique users
users = data.drop_duplicates(USER_COL)[[USER_COL]].reset_index(drop=True)

# Ranking prediction set
ranking_pool = user_item_pairs(
    user_df=users,
    item_df=items,
    user_col=USER_COL,
    item_col=ITEM_COL,
    user_item_filter_df=train,  # remove seen items
    shuffle=True
)

# Rating prediction
predictions = list(itertools.islice(
    saved_model.predict(predict_input_fn(ranking_pool)),
    len(ranking_pool)
))
ranking_pool['prediction'] = [p['predictions'][0] for p in predictions]

for m, fn in RANKING_METRICS.items():
    result = fn(test, ranking_pool, **{**cols, 'k': TOP_K})
    name = "{}@{}".format(m, TOP_K)
    print(name, "=", result)

To see if Hyperdrive found a good hyperparameters, we simply test the model with the hyperparameters used in [TensorFlow's wide-deep learning example](https://github.com/tensorflow/models/tree/master/official/wide_deep) as our baseline. The baseline example 
uses DNN (without the wide model) for [MovieLens example](https://github.com/tensorflow/models/blob/master/official/wide_deep/movielens_main.py).

> Note, this is not 'apples to apples' comparison. For example, TensorFlow's movielens example uses *rating-timestamp* as a numeric feature, but we did not use that here because we think the timestamps are not relevant to the movies' ratings. This comparison is more like to show how Hyperdrive can help to find better hyperparameters without requiring exhaustive efforts in searching through a huge space. 

In [ ]:
OUTPUT_NOTEBOOK = "google_output.ipynb"
OUTPUT_MODEL_DIR = "google_wide_deep_checkpoints"

params = {
    'MOVIELENS_DATA_SIZE': MOVIELENS_DATA_SIZE,
    'MODEL_TYPE': 'deep',
    'NUM_SAMPLES_TO_TRAIN': 20000000,
    'BATCH_SIZE': 256,
    'DNN_OPTIMIZER': 'Adam',
    'DNN_OPTIMIZER_LR': 0.001,
    'DNN_HIDDEN_LAYER_1': 256,
    'DNN_HIDDEN_LAYER_2': 256,
    'DNN_HIDDEN_LAYER_3': 256,
    'DNN_HIDDEN_LAYER_4': 128,
    'DNN_USER_DIM': 16,
    'DNN_ITEM_DIM': 64,
    'DNN_DROPOUT': 0.3,
    'DNN_BATCH_NORM': 0,
    'MODEL_DIR': 'google_wide_deep_checkpoints',
    'METRICS': ['rmse', 'mae', 'rsquared', 'exp_var', 'map', 'ndcg', 'precision', 'recall']
}

pm.execute_notebook(
    "../00_quick_start/wide_deep_model_movielens.ipynb",
    OUTPUT_NOTEBOOK,
    parameters=params,
)

In [ ]:
nb = pm.read_notebook(OUTPUT_NOTEBOOK)
display(nb.dataframe)
display(nb.data)

### Cleanup

In [ ]:
shutil.rmtree(SCRIPT_DIR, ignore_errors=True)
shutil.rmtree(DATA_DIR, ignore_errors=True)
shutil.rmtree(MODEL_DIR, ignore_errors=True)

os.remove(OUTPUT_NOTEBOOK)
shutil.rmtree(OUTPUT_MODEL_DIR, ignore_errors=True)